In [1]:
#importar

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression

In [2]:
#conectar google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#traer documento
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSWcs7d0Hz9D4QsdQrMVoYA1jH7uRiYk2SzPr0AH6gB0FyqphhumdJAM4ga-Ebg9vzfKGmW751pXHJ2/pub?output=csv'
df = pd.read_csv(url)
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 0:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 1:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 2:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 3:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 4:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [7]:
#Limpieza y procesamiento
print('Duplicados:', df.duplicated().sum())
print('Faltan:', df.isna().sum().sum())

Duplicados: 0
Faltan: 0


In [8]:
#copia para la ingenieria
copia_df = df.copy()
copia_df['datetime'] = pd.to_datetime(copia_df['datetime'])
copia_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  int64         
 2   holiday     10886 non-null  int64         
 3   workingday  10886 non-null  int64         
 4   weather     10886 non-null  int64         
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   casual      10886 non-null  int64         
 10  registered  10886 non-null  int64         
 11  count       10886 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(8)
memory usage: 1020.7 KB


In [10]:
# Transforma la columna de fecha y hora en un tipo de fecha hora y utilízalo para crear 3 columnas nuevas en el dataframe que contenga:1. Nombre del mes   2. Nombre del día de la semana   3. Hora del día
copia_df['month'] = copia_df['datetime'].dt.month_name()
copia_df['day'] = copia_df['datetime'].dt.day_name()
copia_df['hour'] = copia_df['datetime'].dt.hour.astype('object')

#    Elimina las columnas “datetime” y “season”.  Estas son ahora redundantes.
copia_df.drop(columns = ['datetime', 'season'], inplace = True)

# Confirmación
copia_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   holiday     10886 non-null  int64  
 1   workingday  10886 non-null  int64  
 2   weather     10886 non-null  int64  
 3   temp        10886 non-null  float64
 4   atemp       10886 non-null  float64
 5   humidity    10886 non-null  int64  
 6   windspeed   10886 non-null  float64
 7   casual      10886 non-null  int64  
 8   registered  10886 non-null  int64  
 9   count       10886 non-null  int64  
 10  month       10886 non-null  object 
 11  day         10886 non-null  object 
 12  hour        10886 non-null  object 
dtypes: float64(3), int64(7), object(3)
memory usage: 1.1+ MB


In [11]:
#Las temperaturas en la columna “temp” y “atemp” son en Celsius. Utiliza .appply() para convertirlas en Fahrenheit.

copia_df[['temp', 'atemp']] = copia_df[['temp', 'atemp']].apply(lambda x: 1.8 * x + 32)

copia_df[['temp', 'atemp']].head()

,temp,atemp
0,49.712,57.911
1,48.236,56.543
2,48.236,56.543
3,49.712,57.911
4,49.712,57.911


In [12]:
#Crea una nueva columna, “temp_variance” que es la diferencia entre “temp” y “atemp”. 

copia_df['temp_variance'] = copia_df['temp'] - copia_df['atemp']
# Elimina la columna “atemp”.
copia_df.drop(columns = 'atemp', inplace = True)
# Confirmación.
copia_df.head()

,holiday,workingday,weather,temp,humidity,windspeed,casual,registered,count,month,day,hour,temp_variance
0,0,0,1,49.712,81,0.0,3,13,16,January,Saturday,0,-8.199
1,0,0,1,48.236,80,0.0,8,32,40,January,Saturday,1,-8.307
2,0,0,1,48.236,80,0.0,5,27,32,January,Saturday,2,-8.307
3,0,0,1,49.712,75,0.0,3,10,13,January,Saturday,3,-8.199
4,0,0,1,49.712,75,0.0,0,1,1,January,Saturday,4,-8.199


In [13]:
#Recuerda eliminar las columnas “casual” y “registered” de ambas versiones antes de realizar una modelización.

copia_df.drop(columns = ['casual', 'registered'], inplace = True)
df.drop(columns = ['casual', 'registered'], inplace = True)

# Confirmación.
display(copia_df.head())
display(df.head())

,holiday,workingday,weather,temp,humidity,windspeed,count,month,day,hour,temp_variance
0,0,0,1,49.712,81,0.0,16,January,Saturday,0,-8.199
1,0,0,1,48.236,80,0.0,40,January,Saturday,1,-8.307
2,0,0,1,48.236,80,0.0,32,January,Saturday,2,-8.307
3,0,0,1,49.712,75,0.0,13,January,Saturday,3,-8.199
4,0,0,1,49.712,75,0.0,1,January,Saturday,4,-8.199


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
0,2011-01-01 0:00:00,1,0,0,1,9.84,14.395,81,0.0,16
1,2011-01-01 1:00:00,1,0,0,1,9.02,13.635,80,0.0,40
2,2011-01-01 2:00:00,1,0,0,1,9.02,13.635,80,0.0,32
3,2011-01-01 3:00:00,1,0,0,1,9.84,14.395,75,0.0,13
4,2011-01-01 4:00:00,1,0,0,1,9.84,14.395,75,0.0,1
